# quantum ml classification

comparing quantum vs classical approaches for binary classification. using qiskit for the quantum stuff.

three models:
1. classical SVM (rbf kernel) - the baseline
2. quantum kernel SVM - quantum feature map + regular svm
3. VQC - variational quantum classifier, fully parameterized circuit

testing on make_moons and iris (2d). these are small datasets so we dont really expect quantum advantage, this is more of a proof of concept.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from pathlib import Path

from src.data_utils import load_moons_dataset, load_iris_2d
from src.quantum_classifier import train_vqc, predict_grid
from src.quantum_kernel import (
    train_quantum_kernel_svm, evaluate_quantum_kernel_svm,
    predict_grid_quantum_kernel
)
from src.classical_baseline import (
    train_classical_svm, evaluate_classical_svm, predict_grid_classical
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True)

print('imports done')

## 1. load the datasets

two datasets, both 2d (= 2 qubits). features scaled to [0, pi] bc the quantum feature map uses them as rotation angles.

In [ ]:
# load both datasets
X_train_moons, X_test_moons, y_train_moons, y_test_moons = load_moons_dataset(n_samples=200)
X_train_iris, X_test_iris, y_train_iris, y_test_iris = load_iris_2d()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(X_train_moons[:, 0], X_train_moons[:, 1],
                c=y_train_moons, cmap='coolwarm', alpha=0.7, edgecolors='k', s=50)
axes[0].set_title('make_moons')
axes[0].set_xlabel('feature 1')
axes[0].set_ylabel('feature 2')

axes[1].scatter(X_train_iris[:, 0], X_train_iris[:, 1],
                c=y_train_iris, cmap='coolwarm', alpha=0.7, edgecolors='k', s=50)
axes[1].set_title('iris (2d)')
axes[1].set_xlabel('sepal length (scaled)')
axes[1].set_ylabel('sepal width (scaled)')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'datasets.png', dpi=150)
plt.show()

print(f"moons: {len(X_train_moons)} train, {len(X_test_moons)} test")
print(f"iris:  {len(X_train_iris)} train, {len(X_test_iris)} test")

## 2. look at the quantum circuits

the ZZFeatureMap encodes our 2d data into a 2-qubit state. the ZZ entangling gates create correlations between qubits based on products of input features which is where the nonlinearity comes from. kind of like a polynomial kernel but quantum.

In [ ]:
from src.quantum_classifier import build_feature_map, build_ansatz

fm = build_feature_map(n_qubits=2, reps=2)
print("=== ZZFeatureMap (2 qubits, 2 reps) ===")
print(fm.decompose().draw(output='text'))

print("\n=== RealAmplitudes ansatz (2 qubits, 3 reps) ===")
ans = build_ansatz(n_qubits=2, reps=3)
print(ans.decompose().draw(output='text'))
print(f"\ntrainable params: {ans.num_parameters}")

## 3. classical SVM baseline

training classical svms first. rbf kernel svm is a strong baseline for 2d classification, gonna be hard to beat.

In [ ]:
# classical svm
classical_svm_moons = train_classical_svm(X_train_moons, y_train_moons, kernel='rbf')
acc_classical_moons, preds_classical_moons = evaluate_classical_svm(
    classical_svm_moons, X_test_moons, y_test_moons
)
print(f"classical SVM on make_moons: {acc_classical_moons:.4f}")

classical_svm_iris = train_classical_svm(X_train_iris, y_train_iris, kernel='rbf')
acc_classical_iris, preds_classical_iris = evaluate_classical_svm(
    classical_svm_iris, X_test_iris, y_test_iris
)
print(f"classical SVM on iris:       {acc_classical_iris:.4f}")

## 4. quantum kernel SVM

quantum kernel approach: compute kernel matrix using quantum circuit, then feed into regular sklearn svm. the kernel value between two points is the state overlap (fidelity) of their encoded quantum states.

note: computing the kernel matrix is slow bc it runs a circuit for every pair of points.

In [ ]:
# quantum kernel svm on make_moons
print("training quantum kernel SVM on make_moons...")
qk_svm_moons, qk_kernel_moons, qk_fm_moons = train_quantum_kernel_svm(
    X_train_moons, y_train_moons, n_qubits=2, reps=2
)
acc_qk_moons, preds_qk_moons = evaluate_quantum_kernel_svm(
    qk_svm_moons, qk_kernel_moons, X_train_moons, X_test_moons, y_test_moons
)
print(f"quantum kernel SVM on make_moons: {acc_qk_moons:.4f}")

In [ ]:
# quantum kernel svm on iris
print("training quantum kernel SVM on iris...")
qk_svm_iris, qk_kernel_iris, qk_fm_iris = train_quantum_kernel_svm(
    X_train_iris, y_train_iris, n_qubits=2, reps=2
)
acc_qk_iris, preds_qk_iris = evaluate_quantum_kernel_svm(
    qk_svm_iris, qk_kernel_iris, X_train_iris, X_test_iris, y_test_iris
)
print(f"quantum kernel SVM on iris: {acc_qk_iris:.4f}")

## 5. variational quantum classifier (VQC)

the VQC is the fully quantum approach - feature map encodes the data, ansatz is the trainable part, optimized with COBYLA. kinda like a neural net but quantum.

this is the slowest one to train.

In [ ]:
# VQC on make_moons - takes a while
print("training VQC on make_moons...")
vqc_moons, obj_vals_moons = train_vqc(
    X_train_moons, y_train_moons,
    n_qubits=2, feature_reps=2, ansatz_reps=3, maxiter=100
)

preds_vqc_moons = vqc_moons.predict(X_test_moons)
acc_vqc_moons = accuracy_score(y_test_moons, preds_vqc_moons)
print(f"VQC on make_moons: {acc_vqc_moons:.4f}")

In [ ]:
# VQC on iris
print("training VQC on iris...")
vqc_iris, obj_vals_iris = train_vqc(
    X_train_iris, y_train_iris,
    n_qubits=2, feature_reps=2, ansatz_reps=3, maxiter=100
)

preds_vqc_iris = vqc_iris.predict(X_test_iris)
acc_vqc_iris = accuracy_score(y_test_iris, preds_vqc_iris)
print(f"VQC on iris: {acc_vqc_iris:.4f}")

## 6. training curves

how the VQC objective changes during training. should go down if the optimizer is working.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(obj_vals_moons, linewidth=1.5, color='#2196F3')
axes[0].set_xlabel('iteration')
axes[0].set_ylabel('objective')
axes[0].set_title('VQC training - make_moons')
axes[0].grid(True, alpha=0.3)

axes[1].plot(obj_vals_iris, linewidth=1.5, color='#FF6B6B')
axes[1].set_xlabel('iteration')
axes[1].set_ylabel('objective')
axes[1].set_title('VQC training - iris')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'training_curves.png', dpi=150)
plt.show()

## 7. decision boundaries

plotting all three classifiers side by side. quantum kernel uses a coarser grid (30x30) bc computing the kernel for every grid point is slow.

In [ ]:
# decision boundaries - make_moons
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
X_all_moons = np.vstack([X_train_moons, X_test_moons])

# classical
xx, yy, Z_classical = predict_grid_classical(classical_svm_moons, X_all_moons)
axes[0].contourf(xx, yy, Z_classical, alpha=0.3, cmap='coolwarm')
axes[0].scatter(X_test_moons[:, 0], X_test_moons[:, 1], c=y_test_moons,
               cmap='coolwarm', edgecolors='k', s=50)
axes[0].set_title(f'classical SVM (acc={acc_classical_moons:.2f})')

# quantum kernel
print("computing quantum kernel decision boundary...")
xx_qk, yy_qk, Z_qk = predict_grid_quantum_kernel(
    qk_svm_moons, qk_kernel_moons, X_train_moons, X_all_moons, resolution=30
)
axes[1].contourf(xx_qk, yy_qk, Z_qk, alpha=0.3, cmap='coolwarm')
axes[1].scatter(X_test_moons[:, 0], X_test_moons[:, 1], c=y_test_moons,
               cmap='coolwarm', edgecolors='k', s=50)
axes[1].set_title(f'quantum kernel SVM (acc={acc_qk_moons:.2f})')

# vqc
print("computing VQC decision boundary...")
xx_vqc, yy_vqc, Z_vqc = predict_grid(vqc_moons, X_all_moons)
axes[2].contourf(xx_vqc, yy_vqc, Z_vqc, alpha=0.3, cmap='coolwarm')
axes[2].scatter(X_test_moons[:, 0], X_test_moons[:, 1], c=y_test_moons,
               cmap='coolwarm', edgecolors='k', s=50)
axes[2].set_title(f'VQC (acc={acc_vqc_moons:.2f})')

for ax in axes:
    ax.set_xlabel('feature 1')
    ax.set_ylabel('feature 2')

plt.suptitle('decision boundaries - make_moons', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'decision_boundaries_moons.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# decision boundaries - iris
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
X_all_iris = np.vstack([X_train_iris, X_test_iris])

xx, yy, Z = predict_grid_classical(classical_svm_iris, X_all_iris)
axes[0].contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
axes[0].scatter(X_test_iris[:, 0], X_test_iris[:, 1], c=y_test_iris,
               cmap='coolwarm', edgecolors='k', s=50)
axes[0].set_title(f'classical SVM (acc={acc_classical_iris:.2f})')

print("computing quantum kernel decision boundary for iris...")
xx_qk, yy_qk, Z_qk = predict_grid_quantum_kernel(
    qk_svm_iris, qk_kernel_iris, X_train_iris, X_all_iris, resolution=30
)
axes[1].contourf(xx_qk, yy_qk, Z_qk, alpha=0.3, cmap='coolwarm')
axes[1].scatter(X_test_iris[:, 0], X_test_iris[:, 1], c=y_test_iris,
               cmap='coolwarm', edgecolors='k', s=50)
axes[1].set_title(f'quantum kernel SVM (acc={acc_qk_iris:.2f})')

print("computing VQC decision boundary for iris...")
xx_vqc, yy_vqc, Z_vqc = predict_grid(vqc_iris, X_all_iris)
axes[2].contourf(xx_vqc, yy_vqc, Z_vqc, alpha=0.3, cmap='coolwarm')
axes[2].scatter(X_test_iris[:, 0], X_test_iris[:, 1], c=y_test_iris,
               cmap='coolwarm', edgecolors='k', s=50)
axes[2].set_title(f'VQC (acc={acc_vqc_iris:.2f})')

for ax in axes:
    ax.set_xlabel('feature 1')
    ax.set_ylabel('feature 2')

plt.suptitle('decision boundaries - iris', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'decision_boundaries_iris.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. results summary

putting it all together

In [ ]:
# bar chart comparison
fig, ax = plt.subplots(figsize=(10, 6))

methods = ['classical SVM\n(rbf)', 'quantum kernel\nSVM', 'VQC']
moons_accs = [acc_classical_moons, acc_qk_moons, acc_vqc_moons]
iris_accs = [acc_classical_iris, acc_qk_iris, acc_vqc_iris]

x = np.arange(len(methods))
width = 0.3

bars1 = ax.bar(x - width/2, moons_accs, width, label='make_moons',
               color='#FF6B6B', alpha=0.8, edgecolor='black', linewidth=0.5)
bars2 = ax.bar(x + width/2, iris_accs, width, label='iris (2d)',
               color='#4ECDC4', alpha=0.8, edgecolor='black', linewidth=0.5)

for bars in [bars1, bars2]:
    for bar in bars:
        h = bar.get_height()
        ax.annotate(f'{h:.2f}', xy=(bar.get_x() + bar.get_width()/2, h),
                   xytext=(0, 3), textcoords='offset points', ha='center', fontsize=11)

ax.set_ylabel('test accuracy')
ax.set_title('quantum vs classical accuracy')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.set_ylim(0.7, 1.05)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'accuracy_comparison.png', dpi=150)
plt.show()

# print results table
print("\n" + "="*55)
print(f"{'method':<25} {'make_moons':>12} {'iris (2d)':>12}")
print("="*55)
print(f"{'classical SVM (rbf)':<25} {acc_classical_moons:>12.4f} {acc_classical_iris:>12.4f}")
print(f"{'quantum kernel SVM':<25} {acc_qk_moons:>12.4f} {acc_qk_iris:>12.4f}")
print(f"{'VQC':<25} {acc_vqc_moons:>12.4f} {acc_vqc_iris:>12.4f}")
print("="*55)

## conclusions

so basically:
- **classical SVM** wins on both datasets. not surprising, these are tiny 2d datasets
- **quantum kernel SVM** is pretty close though. the quantum feature map does a decent job as a kernel
- **VQC** is hardest to train. COBYLA gets stuck in local minima sometimes, maybe a different optimizer would help

the whole point of quantum ml isn't really about beating classical on toy data though. the idea (from havlicek et al) is that quantum kernels could have an advantage on high-dimensional data where classical kernels struggle. this is just the proof of concept for the pipeline.

would be cool to try this on a bigger dataset or with more qubits but the simulation gets really slow.